In [1]:
import numpy as num
from scipy.stats import norm
#from matplotlib import pyplot as mpl, cm
import math as math
#from copy import deepcopy
#from scipy.spatial import distance_matrix
#num.set_printoptions(suppress=True)

In [2]:
def genInstancesNormales(ri=2, re=4, m=10, n=10, iso=500.0, ninst=1, ti=1500.0, te=50.0, 
                 file="./entradas/entrada1.txt", internalNoise=0.1, externalNoise=0.1):
    f = open(file,'w',encoding = 'utf-8')
    f.write("{0:.1f} {1:.1f} {2:} {3:} {4:.1f} {5:}\n".format(ri, re, m+1, n, iso, ninst))
    
    for insta in range(0, ninst):
        internalTemps = num.full(n, ti)
        internalTemps += num.random.normal(0,internalNoise,n)
        
        externalTemps = num.full(n, te)
        externalTemps += num.random.normal(0,externalNoise,n)    
        
        f.write(' '.join(["{:.2f}".format(elem) for elem in internalTemps]))
        f.write(' ')
        f.write(' '.join(["{:.2f}".format(elem) for elem in externalTemps]))
        f.write('\n')
    f.close()

In [32]:
genInstancesNormales()

In [33]:
def genInstancesPeligrosas(ri=2, re=4, m=10, n=10, iso=500.0, ninst=1, ti=1500.0, te=50.0, file="./entradas/entrada1.txt", 
                           internalNoise=0.1, externalNoise=0.1, multDeformacion=100):
    f = open(file,'w',encoding = 'utf-8')
    
    f.write("{0:.1f} {1:.1f} {2:} {3:} {4:.1f} {5:}\n".format(ri, re, m+1, n, iso, ninst))
    
    for insta in range(0, ninst):
        internalTemps = num.full(n, ti)
        internalTemps += num.random.normal(0,internalNoise,n)
        
        externalTemps = num.full(n, te)
        externalTemps += num.random.normal(0,externalNoise,n)
        x = num.linspace(-4,4,n)
        externalTemps += norm.pdf(x, 0, 1)*multDeformacion
        print(externalTemps)
        
        f.write(' '.join(["{:.2f}".format(elem) for elem in internalTemps]))
        f.write(' ')
        f.write(' '.join(["{:.2f}".format(elem) for elem in externalTemps]))
        f.write('\n')
    f.close()


In [34]:
genInstancesPeligrosas()

[50.09955979 50.40935274 53.45964423 66.33124053 86.06937359 86.12889221
 66.51353537 53.41606534 50.25483506 50.06852978]


In [2]:
def linSepInstances(means=[[1,1],[-1,-1]], sigmas=[0.4,0.4], Ns=[100,100]):
    D=len(means[0])

    data = num.random.multivariate_normal(means[0], (sigmas[0]**2)*num.identity(D),Ns[0])
    data = num.append(data, num.random.multivariate_normal(means[1], (sigmas[1]**2)*num.identity(D),Ns[1]),axis=0)
    targets = num.append(num.ones((Ns[0],1)), num.ones((Ns[1],1))*-1, axis=0)
    data = num.append(data,targets,axis=1)
    num.random.shuffle(data)
    return data[:, 0:D], data[:,D:D+1]


In [3]:
def makeCircle(r=1, c=[0,0], sigma=0.1, N=100, D=2):
    noise = num.random.multivariate_normal(num.zeros(D), (sigma**2)*num.identity(D),N)
    data = num.random.uniform(-r,r,(N,1))
    data = num.append(data, num.random.choice([1,-1], (N,1)) * num.sqrt(r**2 - num.square(data)), axis=1)
    data = data + noise + c
    return data

def nonSepInstances(rs=[0.5, 1.2], cs=[[0,0], [0,0]], sigmas=[0.1,0.1], Ns= [100,100]):
    D = len(cs[0])
    data = makeCircle(rs[0], cs[0], sigmas[0], Ns[0], D=D)
    data = num.append(data, makeCircle(rs[1], cs[1], sigmas[1], Ns[1], D=D), axis=0)
    data = num.append(data, num.append(num.ones((Ns[0],1)), (-1)*num.ones((Ns[1],1)), axis=0), axis=1)
    num.random.shuffle(data)
    
    return data[:,0:D], data[:,D:D+1]


### Generacion de casos para evaluar la performance en tiempo de los métodos EG vs LU.

In [10]:
# def generateTestFileForEGvsLU():
#     ninst = 1
#     for n in range(10, 50):
#         for m in range(10, 50):
#             archivo='./testEGvsLU/input/file_n' + str(n) + '_m' + str(m) + '_ninst' + str(ninst) + '.in'
#             genInstancesNormales(ri=4, re=6, m=m, n=n, iso=500, ninst=ninst, ti=1500.0, te=50.0, externalNoise=5.0, file=archivo )

def generateTestFileForEGvsLU(directory, min_n = 10, max_n = 50, min_m = 10, max_m = 50, min_ninst = 1, max_ninst = 1):
    if not os.path.exists(directory):
        os.mkdir(directory)

    for n in range(min_n, max_n+1):
        for m in range(min_m, max_m+1):
            for ninst in range(min_ninst, max_ninst+1):
                archivo= directory + 'file_n' + str(n) + '_m' + str(m) + '_ninst' + str(ninst) + '.in'
                genInstancesNormales(ri=4, re=6, m=m, n=n, iso=500, ninst=ninst, ti=1500.0, te=50.0, externalNoise=5.0, file=archivo )

In [14]:
# generateTestFileForEGvsLU('./testEGvsLU/input/')
# test 2
generateTestFileForEGvsLU('./testEGvsLU/input2/', min_n=16, max_n=16, min_m=8, max_m=8, min_ninst=1, max_ninst=500)
# test 3
generateTestFileForEGvsLU('./testEGvsLU/input3/', min_n=16, max_n=16, min_m=16, max_m=16, min_ninst=1, max_ninst=500)
# test 4
generateTestFileForEGvsLU('./testEGvsLU/input4/', min_n=32, max_n=32, min_m=16, max_m=16, min_ninst=1, max_ninst=200)
# test 5
generateTestFileForEGvsLU('./testEGvsLU/input5/', min_n=32, max_n=32, min_m=32, max_m=32, min_ninst=1, max_ninst=200)

### Benchmarks

In [1]:
import subprocess as sp
import os
import json
from datetime import datetime

now = datetime.now()
currentTime = now.strftime("%Y%m%d_%H%M%S")

pathToExe = "../../build/Release/BenchmarkEGvsLU.exe"
repeat = 10

inputFilesDir = './testEGvsLU/input/'
results = []

for filename in os.listdir(inputFilesDir):
    input_file = os.path.join(inputFilesDir, filename)
    if os.path.isfile(input_file) and input_file.endswith(".in"):
        process = sp.Popen([pathToExe, input_file, str(repeat)], stdout=sp.PIPE)
        (output, err) = process.communicate()
        exitCode = process.wait()
        resultStr = output.decode('utf-8')
        results.append(json.loads(resultStr))

outputFilename = './testEGvsLU/results_{:}.json'.format(currentTime)
with open(outputFilename,'w', encoding='utf-8') as outputFile:
    outputFile.write(json.dumps(results, indent=4))

